In [20]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

def loadFile(filePath):
    inputDir = "./mens-machine-learning-competition-2018"
    df = pd.read_csv(inputDir + "/" + filePath)
    return df

def getTeamId(teamName):
    return teams[(teams['TeamName'] == teamName)]['TeamID'].values[0]

def getTeamName(teamId):
    return teams[(teams['TeamID'] == teamId)]['TeamName'].values[0]

def getTeamWins(df, teamId):
    return df[(df['WTeamID'] == teamId)]

def getTeamLosses(df, teamId):
    return df[(df['LTeamID'] == teamId)]
    
def getGames(df, teamId):
    return teamWins(df, teamId).append(teamLosses(df, teamId))

def getGamesByYear(df, teamId, year):
    gamesWon = df[(df['Season'] == year) & (df['WTeamID'] == teamId)]
    gamesLost = df[(df['Season'] == year) & (df['LTeamID'] == teamId)]
    allGames = gamesWon.append(gamesLost)
    teams = gamesWon['LTeamID'].append(gamesLost['WTeamID'])
    games = {
        'wins': gamesWon,
        'losses': gamesLost,
        'full': allGames,
        'opponents': teams.unique()
    }
    return games

def getPointsPerPossession(teamData):
    offensive = teamData['wins'].loc[:,'WPPPOSS'].append(teamData['losses'].loc[:,'LPPPOSS'])
    defensive = teamData['wins'].loc[:,'LPPPOSS'].append(teamData['losses'].loc[:,'WPPPOSS'])
    retVal = {
        'offensive': {
            'ave': offensive.mean(),
            'std': offensive.std()
        },
        'defensive': {
            'ave': defensive.mean(),
            'std': defensive.std()
           
        }
    }
    return retVal

def displayStats(teamId):
    teamName = getTeamName(teamId)
    teamStats = getTeamStats(teamId)['ppp']
    offensiveAve = teamStats['offensive']['ave']
    offensiveStd = teamStats['offensive']['std']
    defensiveAve = teamStats['defensive']['ave']
    defensiveStd = teamStats['defensive']['std']
    
    print(teamName + ' (' + str(teamId) + '):')
    print('  Offensive:  ave: ' + str(offensiveAve) + '   std: ' + str(offensiveStd))
    print('  Defensive:  ave: ' + str(defensiveAve) + '   std: ' + str(defensiveStd))
    
def addData(df):
    augmented = df.assign(
        WFGPERC = df.WFGM / df.WFGA, 
        WFGPERC3 = df.WFGM3 / df.WFGA3,
        LFGPERC = df.LFGM / df.LFGA,
        LFGPERC3 = df.LFGM3 / df.LFGA3,
        WORPERC = df.WOR / (df.WFGA + df.WFGA3),
        LORPERC = df.LOR / (df.LFGA + df.LFGA3),
        WPOSS = df.WFGA + df.WFGA3 + df.LTO,
        LPOSS = df.LFGA + df.LFGA3 + df.WTO,
        MOV = df.WScore - df.LScore
    )
    second = augmented.assign(
        WPPPOSS = augmented.WScore / augmented.WPOSS,
        LPPPOSS = augmented.LScore / augmented.LPOSS,
        MOP = augmented.WPOSS - augmented.LPOSS
    )

    finalFrame = second.assign(
        MOPPPOSS = second.WPPPOSS - second.LPPPOSS
    )
    
    return finalFrame
    
seasonsFile = "Seasons.csv"
playersFile = "Players_2018.csv"
eventsFile = "Events_2018.csv"
resultsFile = "RegularSeasonCompactResults.csv"
teamsFile= 'Teams.csv'

teams = loadFile(teamsFile)


In [ ]:
year = results[(results['Season'] == 2018)]
#year.head(10)
wins = year[(year['WTeamID'] == utahTeamId)]
losses = year[(year['LTeamID'] == utahTeamId)]
allIndexes = [wins, losses]
all = pd.concat(allIndexes)
boxes=box[(box['Season'] == 2017)]

In [24]:
games = {}

def addTeam(teamId):
    games[str(teamId)] = {}
    
def getTeamStats(teamId):
    return games[str(teamId)]

def addAllGames(teamId):
    if not str(teamId) in games:
        addTeam(teamId)
        team = getTeamStats(teamId)
        teamGames = getGamesByYear(full, teamId, 2017)
        team['games'] = teamGames
        team['ppp'] = getPointsPerPossession(teamGames)
        addAllOpponents()

def addAllOpponents(teamId):
    for oppId in getTeamStats(teamId)['games']['opponents']:
        addAllGames(oppId)
    
def dumpTeamStats(teamId):
    for opponentId in getTeamStats(teamId)['games']['opponents']:
        displayStats(opponentId)
    
utahTeamId = getTeamId('Utah')
arizonaTeamId = getTeamId('Arizona')
uclaTeamId = getTeamId('UCLA')

boxes=loadFile('RegularSeasonDetailedResults.csv')
full = addData(boxes)

addAllGames(utahTeamId)
addAllOpponents(utahTeamId)

addAllGames(uclaTeamId)
addAllOpponents(uclaTeamId)

displayStats(utahTeamId)
print('')
print('Schedule:')
dumpTeamStats(utahTeamId)
print('')
print('UCLA:')
dumpTeamStats(uclaTeamId)




UCLA: 1417
Utah (1428):
  Offensive:  ave: 0.8898025929375443   std: 0.1360295258279601
  Defensive:  ave: 0.7648216348830016   std: 0.11945143221256599

Schedule:
Coppin St (1164):
  Offensive:  ave: 0.7145428267245245   std: 0.17042567056119626
  Defensive:  ave: 0.8406346413414472   std: 0.11047871175623453
UC Riverside (1415):
  Offensive:  ave: 0.7303381241319684   std: 0.1223046217590072
  Defensive:  ave: 0.7733973107284264   std: 0.1559305739723694
Montana St (1286):
  Offensive:  ave: 0.7857878758753063   std: 0.11534539701450959
  Defensive:  ave: 0.8831537759529713   std: 0.14313911329869028
Utah Valley (1430):
  Offensive:  ave: 0.772929023152798   std: 0.1436769670273382
  Defensive:  ave: 0.768604348181814   std: 0.14433017190533115
Prairie View (1341):
  Offensive:  ave: 0.7506488085374262   std: 0.1418374574826777
  Defensive:  ave: 0.8940151660570288   std: 0.18537969996094325
Hawaii (1218):
  Offensive:  ave: 0.7530043607727018   std: 0.14906392246231467
  Defensive: 